In [ ]:
# you create geojson on qgis in advance

In [67]:
from shapely.geometry import LineString
from shapely.ops import transform
from pyproj import Transformer
import utils

get_alt = utils.get_get_alt('./data/_hakone.vrt')
to_meter = Transformer.from_crs('EPSG:4612', 'EPSG:2451', always_xy=True).transform

def get_cartgraphic_degrees(geoms, height, timetable):
    def get_distances(geom):
        lengths = [LineString(geom.coords[i:i+2]).length for i in range(len(geom.coords) - 1)]
        return [0] + [sum(lengths[:i]) for i in range(1, len(lengths) + 1)]

    def get_section(geom, height, start, end):
        dists = get_distances(transform(to_meter, geom))
        fsec = lambda d: (end - start) * (d / dists[-1]) + start
        return [(fsec(d), x, y, get_alt(x, y) + height) for d, (x, y) in zip(dists, geom.coords)]

    sections = [get_section(geom, height, start, end) for geom, (start, end) in zip(geoms, timetable)]
    return [a for section in sections for pos in section for a in pos]

In [37]:
def get_czml_header(name):
    return {
        'id': 'document',
        'name': name,
        'version': '1.0'
    }

def get_czml_body(aid, name, description, epoch, cartgraphic_degrees):
    return {
        'id': aid,
        'name': name,
        'description': description,
        'position': {
            'epoch': epoch.strftime('%Y-%m-%dT%H:%M:%S+09'),
            'cartographicDegrees': cartgraphic_degrees
        },
        'orientation': {
            'velocityReference': "%s#position" % aid
        },
        'forwardExtrapolationType': 'None',
        'backwardExtrapolationType': 'None',
        'model': {
            'gltf': 'data/vehicles/pedestrian.glb'
        }
    }

In [86]:
import random as rd

TIME_DISTANCES = [
     7*60, # 箱根湯本→郷土資料館
    10*60, # 郷土資料館→早雲寺
     8*60, # 早雲寺→正眼寺
    17*60, # 正眼寺→箱根観音
    11*60, # 箱根観音→玉簾の滝
    17*60, # 玉簾の滝→湯場・熊野神社
     8*60, # 湯場・熊野神社→箱根湯本
]

def get_timetable():
    timetables = []
    for td in TIME_DISTANCES:
        last = timetables[-1][1] if len(timetables) else 0
        start = last + rd.randint(60, 500)
        timetables.append([start, start + td])
    return timetables

[[80, 500],
 [994, 1594],
 [1746, 2226],
 [2443, 3463],
 [3715, 4375],
 [4784, 5804],
 [6000, 6480]]

In [92]:
import json
from datetime import datetime as dt, timedelta as td
import random as rd
import geopandas as gpd
from pyproj import Transformer
from shapely.ops import transform, substring

# make pedestrian czml

TIME_DISTANCES = [
     7*60, # 箱根湯本→郷土資料館
    10*60, # 郷土資料館→早雲寺
     8*60, # 早雲寺→正眼寺
    17*60, # 正眼寺→箱根観音
    11*60, # 箱根観音→玉簾の滝
    17*60, # 玉簾の滝→湯場・熊野神社
     8*60, # 湯場・熊野神社→箱根湯本
]
SEEDS = [
    [6, 900], [7, 600], [8, 600], [9, 300], [10, 180], [11, 180],
]
HEIGHT = 40.0222
ELLIPSOID = {
    'radii': { 'cartesian': [1, 1, 1.8] },
    'material': { 'solidColor': { 'color': { 'rgba': [216, 216, 216, 0xff] } } },
    'shadows': 'ENABLED'
}
DESCRIPTION = 'Aコース 早雲寺と石畳の旧街道 玉簾の滝<br>移動時間 約75分<br>箱根湯本駅〜(7分)〜郷土資料館〜(10分)〜早雲寺〜(2分)<br>〜白山神社〜(6分)〜正眼寺〜(11分)〜箱根観音〜(11分)<br>〜玉簾の滝〜(14分)〜湯場・熊野神社〜(8分)〜箱根湯本駅'

get_alt = utils.get_get_alt('./data/_hakone.vrt')
to_meter = Transformer.from_crs('EPSG:4612', 'EPSG:2451', always_xy=True).transform
to_latlon = Transformer.from_crs('EPSG:2451', 'EPSG:4612', always_xy=True).transform

def get_secs(hour, interval):
    intervals = [rd.normalvariate(interval, interval / 3) for _ in range(120)]
    return [a + hour * 3600 for i, _ in enumerate(intervals) if (a:=sum(intervals[:i])) < 3600]

def get_timetable():
    timetables = []
    for td in TIME_DISTANCES:
        last = timetables[-1][1] if len(timetables) else 0
        start = last + rd.randint(60, 500)
        timetables.append([start, start + td])
    return timetables

secs = sum([get_secs(*seed) for seed in SEEDS], [])
geoms = gpd.read_file('./data/_rdcl_w_tmp.geojson').sort_values('number').geometry
n_geoms = [substring(transform(to_latlon, transform(to_meter, geom).parallel_offset(1, 'right')), 1, 0, normalized=True) for geom in geoms]
cart_degs = get_cartgraphic_degrees(n_geoms, HEIGHT, get_timetable())
packets = [get_czml_header('歩行者')]
n = dt.now()
for i, sec in enumerate(secs):
    epoch = dt.strptime(f'{n.year}-{n.month}-{n.day}', '%Y-%m-%d') + td(seconds=sec)
    packets += [get_czml_body(f'{i}_p', '箱根湯本ぶらぶらお散歩コース', DESCRIPTION, epoch, cart_degs)]
open('./dst/pedestrian_a_czml.json', 'w').write(json.dumps(packets, indent=2, ensure_ascii=False))

7446418

In [99]:
import json
from datetime import datetime as dt, timedelta as td
import random as rd
import geopandas as gpd
from pyproj import Transformer
from shapely.ops import transform, substring

# make pedestrian czml (reverse)

TIME_DISTANCES = [
     8*60, # 湯場・熊野神社→箱根湯本
    17*60, # 玉簾の滝→湯場・熊野神社
    11*60, # 箱根観音→玉簾の滝
    17*60, # 正眼寺→箱根観音
     8*60, # 早雲寺→正眼寺
    10*60, # 郷土資料館→早雲寺
     7*60, # 箱根湯本→郷土資料館
]
SEEDS = [
    [6, 900], [7, 600], [8, 600], [9, 300], [10, 180], [11, 180],
]
HEIGHT = 40.0222
ELLIPSOID = {
    'radii': { 'cartesian': [1, 1, 1.8] },
    'material': { 'solidColor': { 'color': { 'rgba': [216, 216, 216, 0xff] } } },
    'shadows': 'ENABLED'
}
DESCRIPTION = 'Aコース 早雲寺と石畳の旧街道 玉簾の滝（逆回り）<br>移動時間 約75分<br>箱根湯本駅〜(8分)〜湯場・熊野神社〜(14分)〜玉簾の滝〜(11分)<br>〜箱根観音〜(11分)〜正眼寺〜(6分)〜白山神社〜(2分)<br>〜早雲寺〜(10分)〜郷土資料館〜(7分)〜箱根湯本駅'

get_alt = utils.get_get_alt('./data/_hakone.vrt')
to_meter = Transformer.from_crs('EPSG:4612', 'EPSG:2451', always_xy=True).transform
to_latlon = Transformer.from_crs('EPSG:2451', 'EPSG:4612', always_xy=True).transform

def get_secs(hour, interval):
    intervals = [rd.normalvariate(interval, interval / 3) for _ in range(120)]
    return [a + hour * 3600 for i, _ in enumerate(intervals) if (a:=sum(intervals[:i])) < 3600]

def get_timetable():
    timetables = []
    for td in TIME_DISTANCES:
        last = timetables[-1][1] if len(timetables) else 0
        start = last + rd.randint(60, 500)
        timetables.append([start, start + td])
    return timetables

secs = sum([get_secs(*seed) for seed in SEEDS], [])
geoms = gpd.read_file('./data/_rdcl_w_tmp.geojson').sort_values('number', ascending=False).geometry
geoms = [substring(geom, 1, 0, normalized=True) for geom in geoms]
n_geoms = [substring(transform(to_latlon, transform(to_meter, geom).parallel_offset(1, 'right')), 1, 0, normalized=True) for geom in geoms]
cart_degs = get_cartgraphic_degrees(n_geoms, HEIGHT, get_timetable())
packets = [get_czml_header('歩行者')]
n = dt.now()
for i, sec in enumerate(secs):
    epoch = dt.strptime(f'{n.year}-{n.month}-{n.day}', '%Y-%m-%d') + td(seconds=sec)
    packets += [get_czml_body(f'{i}_p', '箱根湯本ぶらぶらお散歩コース', DESCRIPTION, epoch, cart_degs)]
open('./dst/pedestrian_b_czml.json', 'w').write(json.dumps(packets, indent=2, ensure_ascii=False))

6825591